In [1]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

In [3]:
# use existing cluster
from azureml.core import Workspace
ws = Workspace.from_config()

In [5]:
# create new cluster
cluster_name = 'pandasaurus'
cluster = ''
try:
    configs = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_V2', max_nodes=2)
    cluster = ComputeTarget.create(workspace=ws, name=cluster_name, provisioning_configuration=configs)
    cluster.wait_for_completion(show_output=True)
except ComputeTargetException as ex:
    print(ex)


Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [16]:
# Hyperdrive-> Command Line Applications -> Assuming we had command line app
# Hyperparameter TUNING package 
# EARLY STOPPING 
from azureml.core import Experiment
from azureml.train.sklearn import SKLearn # this is how to create an Hyperdrive ESTIMATOR 
# hp selection policy
from azureml.train.hyperdrive import GridParameterSampling
from azureml.train.hyperdrive import RandomParameterSampling
# early stopping policy
from azureml.train.hyperdrive import BanditPolicy
# PrimaryMetricGoal -> this the desired metric evaluation 
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal, choice, uniform
from azureml.widgets import RunDetails



In [17]:
#param = {} # algos object to tune HPS with
# Linear Regression as example
# L2 -> try diff values for it

# DEFINING the SEARCH SPACE 

param = RandomParameterSampling({
    '--l2':choice(0.01,0.02,0.001),
    '--lr':uniform(0.05, 0.1)
})

param = GridParameterSampling({
    '--l2':choice(0.01,0.02,0.001),
    '--lr':choice(0.05, 0.1, 0.001)
})


# CAN also be used to TEST MODEL Performance 

# IMPORTANT-> your command line application has to SUPPORT an l2 and parse it via ARGPARSE 
# Train.py --l2 0.01
# Train.py --l2 0.02
# Train.py --l2 0.001




In [20]:
# create SKLEARN hyperdrive estimator
exp_folder = 'assume current working directory'
dataset = ws.datasets.get('bca')
script = 'command line application path'
estimator = SKLearn(source_directory=exp_folder,
 inputs = [dataset.as_named_input('bca')] 
 ,pip_packages=['azureml-sdk'],
 entry_script=script,
 compute_target= cluster)

In [21]:
# Use this-> Experiment -> LOG the runs 
hd_config = HyperDriveConfig(estimator=estimator, 
hyperparameter_sampling=param,
policy = None,
primary_metric_name='AUC',
primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
max_total_runs=12,
max_concurrent_runs=6)
# max total runs/max_concurrent_runs -> NO. of Parallel executions 
exp = Experiment(workspace=ws, name='hyperdrive_bca')
run = exp.submit(config=hd_config)


FileNotFoundError: [Errno 2] No such file or directory: 'assume current working directory'

In [22]:
# Execute the RUN above 
RunDetails(run).show()
run.wait_for_completion()

NameError: name 'run' is not defined